In [1]:
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')
#!ls drive/'My Drive/YYY_deep_project_YYY'

In [2]:
from collections import defaultdict

def sequences_to_dicts(sequences):
    """
    Creates word_to_idx and idx_to_word dictionaries for a list of sequences.
    """
    # A bit of Python-magic to flatten a nested list
    flatten = lambda l: [item for sublist in l for item in sublist]

    # Flatten the dataset
    all_words = flatten(sequences)

    # Count number of word occurences
    word_count = defaultdict(int)
    for word in flatten(sequences):
        word_count[word] += 1

    # Sort by frequency
    word_count = sorted(list(word_count.items()), key=lambda l: -l[1])

    # Create a list of all unique words
    unique_words = [item[0] for item in word_count]

    # Add UNK token to list of words
    unique_words.append('UNK')

    # Count number of sequences and number of unique words
    num_sentences, vocab_size = len(sequences), len(unique_words)

    # Create dictionaries so that we can go from word to index and back
    # If a word is not in our vocabulary, we assign it to token 'UNK'
    word_to_idx = defaultdict(lambda: vocab_size-1)
    idx_to_word = defaultdict(lambda: 'UNK')

    # Fill dictionaries
    for idx, word in enumerate(unique_words):
        word_to_idx[word] = idx
        idx_to_word[idx] = word

    return word_to_idx, idx_to_word, num_sentences, vocab_size

In [3]:
def get_sequence(infile):

    while True:

        header = infile.readline()
        sequence = infile.readline()

        pdb = header[1:5]

        if not header or not sequence or set(sequence) == {'X'}:
            return
        
        yield header.strip()[1:], sequence.strip(), pdb

In [4]:
sequences = []
seq_to_pdb = {}
count = 0
with open('all_heavy.fasta') as infile:

        for header, sequence, pdb in get_sequence(infile):
            #if count < 500:
            sequences.append(list(sequence))
                #count += 1
            
            seq_to_pdb[sequence] = pdb

In [6]:
import pandas as pd

df = pd.read_csv('sabdab_summary_all-2.tsv', sep='\t')
df = df[['pdb', 'affinity']]
df

,pdb,affinity
0,5m2j,1.3000000000000002e-10
1,6fe4,9.6e-09
2,7jmo,None
3,6ch9,None
4,4o51,None
...,...,...
8562,5ukq,None
8563,6ejm,8.6e-10
8564,5bk0,None
8565,3vi3,None


In [7]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        # Return the size of the dataset
        return len(self.targets)

    def __getitem__(self, index):
        # Retrieve inputs and targets at the given index
        X = self.inputs[index]
        y = self.targets[index]

        return X, y

def create_datasets(sequences, dataset_class, p_train=0.8, p_val=0.1, p_test=0.1):
    
    #def seq_handler(sequences, p, start, end):
    #   num = int(len(sequences)*p)
    #   seq_part = sequences
    
    # Define partition sizes
    num_train = int(len(sequences)*p_train)
    num_val = int(len(sequences)*p_val)
    num_test = int(len(sequences)*p_test)

    # Split sequences into partitions
    sequences_train = sequences[:num_train-1]
    sequences_val = sequences[num_train:num_train+num_val-1]
    sequences_test = sequences[-num_test:-1]

    target_seqs_train = [seq[1:] for seq in sequences_train]
    target_seqs_val = [seq[1:] for seq in sequences_val]
    target_seqs_test = [seq[1:] for seq in sequences_test]


    input_train = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in sequences_train] for x in sublist]
    input_val = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in sequences_val] for x in sublist]
    input_test = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in sequences_test] for x in sublist]


    target_train = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in target_seqs_train] for x in sublist]
    target_val = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in target_seqs_val] for x in sublist]
    target_test = [x for sublist in [['<sos>'] + list(seq)+['<eos>'] for seq in target_seqs_test] for x in sublist]


    def get_inputs_targets_from_sequences(sequences):
        # Define empty lists
        inputs, targets = [], []
        
        # Append inputs and targets s.t. both lists contain L-1 words of a sentence of length L
        # but targets are shifted right by one so that we can predict the next word
        for sequence in sequences:
            inputs.append(sequence[:-1])
            targets.append(sequence[1:])

            # We want to predict e.g. last 1/3 of sequence based on first 2/3
            #l = len(sequence) - len(sequence[:int(3*len(sequence)/4)])
            #inputs.append(sequence[:int(3*len(sequence)/4)])
            #targets.append(sequence[l:])
            
        return inputs, targets

    # Get inputs and targets for each partition
    inputs_train, targets_train = get_inputs_targets_from_sequences(sequences_train)
    inputs_val, targets_val = get_inputs_targets_from_sequences(sequences_val)
    inputs_test, targets_test = get_inputs_targets_from_sequences(sequences_test)

    # Create datasets
    training_set = dataset_class(inputs_train, targets_train)
    validation_set = dataset_class(inputs_val, targets_val)
    test_set = dataset_class(inputs_test, targets_test)

    #return training_set, validation_set, test_set

    return input_train, input_val, input_test, target_train, target_val, target_test, training_set, validation_set, test_set
    

input_train, input_val, input_test, target_train, target_val, target_test, training_set, validation_set, test_set = create_datasets(sequences, Dataset)


In [8]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [9]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [14]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("spacy", "en"),
                           init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
#TEXT = torchtext.data.Field(tokenize=get_tokenizer("spacy", "en_core_web_lg"),
#                            init_token='<sos>',
#                            eos_token='<eos>',
#                            lower=True)
#train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
train_txt, val_txt, test_txt = training_set, validation_set, test_set

TEXT.build_vocab(train_txt)
print(TEXT.vocab.stoi)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    #print(data.examples[0].text)
    data = TEXT.numericalize([data])
    
    #data = TEXT.numericalize([data[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(input_train, batch_size)
val_data = batchify(input_val, eval_batch_size)
test_data = batchify(input_test, eval_batch_size)

#print(train_txt)
print(training_set)
train_data.shape
test_data.shape

/Users/jonas/miniconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fefdca5c910>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'S': 4, 'T': 5, 'G': 6, 'V': 7, 'L': 8, 'A': 9, 'P': 10, 'K': 11, 'Y': 12, 'Q': 13, 'D': 14, 'N': 15, 'E': 16, 'F': 17, 'R': 18, 'W': 19, 'I': 20, 'C': 21, 'H': 22, 'M': 23, 'X': 24})


torch.Size([8862, 10])

In [15]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [16]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
lr = 5.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

model.to(device)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)

        data, targets = data.to(device), targets.to(device)
        
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)


In [49]:
best_val_loss = float("inf")
epochs = 50 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

| epoch   1 |   200/ 1002 batches | lr 5.50 | ms/batch  8.00 | loss  3.72 | ppl    41.33
| epoch   1 |   400/ 1002 batches | lr 5.50 | ms/batch  7.62 | loss  2.49 | ppl    12.03
| epoch   1 |   600/ 1002 batches | lr 5.50 | ms/batch  7.69 | loss  2.37 | ppl    10.67
| epoch   1 |   800/ 1002 batches | lr 5.50 | ms/batch  7.68 | loss  2.31 | ppl    10.09
| epoch   1 |  1000/ 1002 batches | lr 5.50 | ms/batch  7.66 | loss  2.30 | ppl     9.93
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  8.25s | valid loss  2.11 | valid ppl     8.27
-----------------------------------------------------------------------------------------
| epoch   2 |   200/ 1002 batches | lr 4.96 | ms/batch  7.86 | loss  2.25 | ppl     9.47
| epoch   2 |   400/ 1002 batches | lr 4.96 | ms/batch  7.71 | loss  2.18 | ppl     8.88
| epoch   2 |   600/ 1002 batches | lr 4.96 | ms/batch  7.74 | loss  2.16 | ppl     8.70
| epoch   2 |   800/ 1002 batches 

| epoch  13 |   200/ 1002 batches | lr 2.82 | ms/batch  7.96 | loss  1.85 | ppl     6.35
| epoch  13 |   400/ 1002 batches | lr 2.82 | ms/batch  7.76 | loss  1.81 | ppl     6.13
| epoch  13 |   600/ 1002 batches | lr 2.82 | ms/batch  7.69 | loss  1.81 | ppl     6.12
| epoch  13 |   800/ 1002 batches | lr 2.82 | ms/batch  7.73 | loss  1.80 | ppl     6.06
| epoch  13 |  1000/ 1002 batches | lr 2.82 | ms/batch  7.71 | loss  1.82 | ppl     6.19
-----------------------------------------------------------------------------------------
| end of epoch  13 | time:  8.29s | valid loss  1.67 | valid ppl     5.31
-----------------------------------------------------------------------------------------
| epoch  14 |   200/ 1002 batches | lr 2.68 | ms/batch  7.85 | loss  1.83 | ppl     6.21
| epoch  14 |   400/ 1002 batches | lr 2.68 | ms/batch  7.66 | loss  1.79 | ppl     6.01
| epoch  14 |   600/ 1002 batches | lr 2.68 | ms/batch  7.64 | loss  1.79 | ppl     6.01
| epoch  14 |   800/ 1002 batches 

| epoch  25 |   200/ 1002 batches | lr 1.53 | ms/batch  7.76 | loss  1.69 | ppl     5.40
| epoch  25 |   400/ 1002 batches | lr 1.53 | ms/batch  7.81 | loss  1.65 | ppl     5.23
| epoch  25 |   600/ 1002 batches | lr 1.53 | ms/batch  8.00 | loss  1.66 | ppl     5.26
| epoch  25 |   800/ 1002 batches | lr 1.53 | ms/batch  7.78 | loss  1.64 | ppl     5.17
| epoch  25 |  1000/ 1002 batches | lr 1.53 | ms/batch  7.80 | loss  1.67 | ppl     5.32
-----------------------------------------------------------------------------------------
| end of epoch  25 | time:  8.37s | valid loss  1.54 | valid ppl     4.65
-----------------------------------------------------------------------------------------
| epoch  26 |   200/ 1002 batches | lr 1.45 | ms/batch  7.93 | loss  1.68 | ppl     5.37
| epoch  26 |   400/ 1002 batches | lr 1.45 | ms/batch  7.84 | loss  1.65 | ppl     5.19
| epoch  26 |   600/ 1002 batches | lr 1.45 | ms/batch  7.79 | loss  1.65 | ppl     5.20
| epoch  26 |   800/ 1002 batches 

| epoch  37 |   200/ 1002 batches | lr 0.82 | ms/batch  7.84 | loss  1.61 | ppl     5.00
| epoch  37 |   400/ 1002 batches | lr 0.82 | ms/batch  8.12 | loss  1.57 | ppl     4.83
| epoch  37 |   600/ 1002 batches | lr 0.82 | ms/batch  7.93 | loss  1.58 | ppl     4.87
| epoch  37 |   800/ 1002 batches | lr 0.82 | ms/batch  7.79 | loss  1.56 | ppl     4.76
| epoch  37 |  1000/ 1002 batches | lr 0.82 | ms/batch  7.95 | loss  1.60 | ppl     4.95
-----------------------------------------------------------------------------------------
| end of epoch  37 | time:  8.46s | valid loss  1.48 | valid ppl     4.40
-----------------------------------------------------------------------------------------
| epoch  38 |   200/ 1002 batches | lr 0.78 | ms/batch  7.90 | loss  1.60 | ppl     4.97
| epoch  38 |   400/ 1002 batches | lr 0.78 | ms/batch  7.79 | loss  1.58 | ppl     4.83
| epoch  38 |   600/ 1002 batches | lr 0.78 | ms/batch  7.70 | loss  1.58 | ppl     4.86
| epoch  38 |   800/ 1002 batches 

| epoch  49 |   200/ 1002 batches | lr 0.45 | ms/batch  8.60 | loss  1.56 | ppl     4.77
| epoch  49 |   400/ 1002 batches | lr 0.45 | ms/batch  8.17 | loss  1.53 | ppl     4.62
| epoch  49 |   600/ 1002 batches | lr 0.45 | ms/batch  8.07 | loss  1.54 | ppl     4.67
| epoch  49 |   800/ 1002 batches | lr 0.45 | ms/batch  8.16 | loss  1.52 | ppl     4.57
| epoch  49 |  1000/ 1002 batches | lr 0.45 | ms/batch  8.11 | loss  1.56 | ppl     4.74
-----------------------------------------------------------------------------------------
| end of epoch  49 | time:  8.79s | valid loss  1.46 | valid ppl     4.31
-----------------------------------------------------------------------------------------
| epoch  50 |   200/ 1002 batches | lr 0.42 | ms/batch  8.22 | loss  1.56 | ppl     4.77
| epoch  50 |   400/ 1002 batches | lr 0.42 | ms/batch  8.09 | loss  1.53 | ppl     4.62
| epoch  50 |   600/ 1002 batches | lr 0.42 | ms/batch  8.08 | loss  1.54 | ppl     4.66
| epoch  50 |   800/ 1002 batches 

In [19]:
#orch.save(model,'C:/Users/jonas/Desktop/deep_project/models/model.py')
# Model class must be defined somewhere
model = torch.load("models/model.py", map_location=torch.device('cpu'))
model.eval()

best_model = model

In [20]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  1.33 | test ppl     3.79


In [21]:
dict(TEXT.vocab.stoi)

{'<unk>': 0,
 '<pad>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'S': 4,
 'T': 5,
 'G': 6,
 'V': 7,
 'L': 8,
 'A': 9,
 'P': 10,
 'K': 11,
 'Y': 12,
 'Q': 13,
 'D': 14,
 'N': 15,
 'E': 16,
 'F': 17,
 'R': 18,
 'W': 19,
 'I': 20,
 'C': 21,
 'H': 22,
 'M': 23,
 'X': 24}

In [34]:
idx_to_letter = {value:key for key,value in dict(TEXT.vocab.stoi).items() if isinstance(key, str) }
ntokens = len(idx_to_letter)
idx_to_letter

{0: '<unk>',
 1: '<pad>',
 2: '<sos>',
 3: '<eos>',
 4: 'S',
 5: 'T',
 6: 'G',
 7: 'V',
 8: 'L',
 9: 'A',
 10: 'P',
 11: 'K',
 12: 'Y',
 13: 'Q',
 14: 'D',
 15: 'N',
 16: 'E',
 17: 'F',
 18: 'R',
 19: 'W',
 20: 'I',
 21: 'C',
 22: 'H',
 23: 'M',
 24: 'X'}

In [57]:
def sample_categorical(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome log-probabilities
    :param temperature: Sampling temperature. 1.0 follows the given distribution,
        0.0 returns the maximum probability element.
    :return: The index of the sampled element.
    """

    if temperature == 0.0:
        return lnprobs.argmax()
    p = F.softmax(lnprobs / temperature, dim=1)

    #print("softmaxed probs:", p)
    
    return dist.Categorical(p).sample()

def sample_sentence(model, query, max_len = 140, temperature=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    while len(query) < max_len:
        query_tensor = batchify(query, len(query))
        src_mask = model.generate_square_subsequent_mask(len(query_tensor)).to(device)
        
        output = model(query_tensor, src_mask).view(-1, ntokens)
        
        #output = model(query_tensor, src_mask)
        
        #print("view(-1)", model(query_tensor, src_mask).view(-1, ntokens))
        #output, _     = model(query_.unsqueeze(0).to(device))
        
        #print("model output:", output)
        
        #next_char_idx = sample_categorical(output[0, :, len(query)-1], temperature) #0.5
        
        next_char_idx = sample_categorical(output, temperature) #0.5
        
        #print(output[0, :, len(query_tensor) - 1])
        #print(output)           #print(next_char_idx)
        
        #print(next_char_idx)

        #if next_char_idx <= 1:
            # query += "*"
        #    break

        #query += [str(chr(int(next_char_idx[0][0])))]

        query += [idx_to_letter[int(next_char_idx[-1])]]
        
    
    return query


In [58]:
import torch.distributions as dist

#dat = batchify(['A'], 1)

#src_mask = model.generate_square_subsequent_mask(dat.size(0)).to(device)
#output = model(dat, src_mask)
#sample_categorical(output,temperature=0.5)

sample_sentence(model, ['<sos>'], max_len = 5, temperature=0.5)


['<sos>', 'Q', 'S', 'S', 'S']

In [76]:


ntokens = len(TEXT.vocab.stoi)

#dat = batchify(['<sos>'], 1)

sample = sample_sentence(model, ['<sos>'], max_len = 200, temperature=0.9)
"".join(sample)

'<sos>QGGTFADTVRLVQLSGYWGAPKVNHH<eos><sos>QSLDKSGLRGYFPVPGRHHPQLSAQGSGGWGTVQLVDTAPSGGFESLKNTPSLQKGR<eos><sos>QQTYGQSSAVQEVNYQGCNH<eos><sos>QPASIYFPLTWFPLAVNVSCVAVWGTYCGREPEWIDTPGGLHGGSVSLGNNYKSGKQSLGLVLAKKVIRESTSAKSSLFNDWVHHHHHHH'